In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Load Markdown

Using `MarkdownNodeParser` for that

In [3]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.file import FlatReader
from pathlib import Path

md_docs = FlatReader().load_data(Path("./data/optisol_info.md"))

parser = MarkdownNodeParser()
nodes = parser.get_nodes_from_documents(md_docs)

In [5]:
len(nodes)

38

### Ingestion with Qdrant

In [6]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [7]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

def create_index(nodes, colection_name):
    vector_store = QdrantVectorStore(colection_name, client=client)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

In [8]:
create_index(nodes, "optisol_first")

### Index to Retriever

In [9]:
vector_store = QdrantVectorStore("optisol_first", client=client)
index = VectorStoreIndex.from_vector_store(vector_store)

In [10]:
index

In [11]:
retriever = index.as_retriever()
returned_nodes = retriever.retrieve("Who is the current Chief Marketing Officer (CMO) of OptiSol Technologies, and what is their background?")

In [13]:
for node in returned_nodes:
    print(node.text)

Leadership Team

Jane Doe, CEO:  
Jane has over 20 years of experience in the IT industry, having held various leadership positions at top technology firms before founding OptiSol Technologies. She is known for her visionary leadership and commitment to driving innovation.

John Smith, CFO:  
John brings extensive financial expertise to OptiSol, with a background in corporate finance and investment banking. He oversees the company's financial operations and strategic investments.

Maria Gonzalez, CTO:  
Maria leads the technology strategy at OptiSol, ensuring the company stays at the forefront of technological advancements. She has a strong background in software development and AI technologies.

David Lee, COO:  
David is responsible for the day-to-day operations at OptiSol, focusing on optimizing processes and improving operational efficiency. He has a wealth of experience in operational management and process improvement.

Emma Brown, CMO:  
Emma oversees all marketing and communica

In [14]:
chat_engine = index.as_chat_engine()

In [15]:
response = chat_engine.query("Who is the current Chief Marketing Officer (CMO) of OptiSol Technologies, and what is their background?")

In [18]:
source_nodes = response.source_nodes

In [20]:
for node in source_nodes:
    print(node.text)

Leadership Team

Jane Doe, CEO:  
Jane has over 20 years of experience in the IT industry, having held various leadership positions at top technology firms before founding OptiSol Technologies. She is known for her visionary leadership and commitment to driving innovation.

John Smith, CFO:  
John brings extensive financial expertise to OptiSol, with a background in corporate finance and investment banking. He oversees the company's financial operations and strategic investments.

Maria Gonzalez, CTO:  
Maria leads the technology strategy at OptiSol, ensuring the company stays at the forefront of technological advancements. She has a strong background in software development and AI technologies.

David Lee, COO:  
David is responsible for the day-to-day operations at OptiSol, focusing on optimizing processes and improving operational efficiency. He has a wealth of experience in operational management and process improvement.

Emma Brown, CMO:  
Emma oversees all marketing and communica